In [203]:
import numpy as np
from scipy.spatial.transform import Rotation

In [199]:
dataset = np.load('demonstrations/energy_rudd_run001.npz')

In [36]:
def get_wind(z):
        wind = np.array([0,0,0], dtype=float)
        shear_top = 15.0
        wind_grad = -1.0
        wind[1] = wind_grad * max(min(z, shear_top), 0.0)
        return wind

In [141]:
def get_ground_velocity(obs):
    z = obs[0]
    airspeed = obs[1]
    yaw = obs[2]
    pitch = obs[3]
    roll = obs[4]

    I_v_wind = np.zeros((3))
    I_v_wind = get_wind(z)

    B_v_wind = np.zeros((3))
    rot = Rotation.from_euler('ZYX', [yaw, pitch, roll])
    
    B_v_wind = rot.apply(I_v_wind)

    B_v_air = np.zeros((B_v_wind.shape))
    B_v_air[0] = airspeed

    return B_v_air + B_v_wind

In [150]:
def get_ground_angular_velocity(prev_obs, obs):
    z = obs[0]
    airspeed = obs[1]
    yaw   = obs[2]
    pitch = obs[3]
    roll  = obs[4]
    prev_yaw   = prev_obs[2]
    prev_pitch = prev_obs[3]
    prev_roll  = prev_obs[4]
    time_step = 0.3

    I_w = np.zeros((3))
    B_w = np.zeros((3))
    I_w = np.array([(yaw - prev_yaw)/time_step,
                    (pitch - prev_pitch)/time_step,
                    (roll - prev_roll)/time_step])

    rot = Rotation.from_euler('ZYX', [yaw, pitch, roll])

    B_w = rot.apply(I_w)
    
    return B_w

In [157]:
def get_net_cg_forces(prev_obs, obs, actions):
    B_F_a1 = np.array([[0.0, -0.009, -0.007],
                      [-0.003, 0.0, 0.08],
                      [0.0, -0.078, 0.0]])
    
    B_F_a2 = np.array([[-0.056, 0.026, -0.001, -0.001, 0.0, 0.0, -0.005],
                       [0.0, -0.296, 0.0, 0.0, 0.0, 0.034, 0.0],
                       [-1.892, 0.0, -0.017, -0.017, -0.027, 0.0, 0.038]])
    g = 9.81
    m = 1.0
    alpha = 0.0
    beta = 0.0

    p = actions[1]
    q = actions[0]
    r = actions[2]

    act = np.array([alpha, beta, p, q, r, 0.0, 1.0])
    airspeed = obs[1]

    I_F_g = np.array([0.0, 0.0, m*g])

    B_w = get_ground_angular_velocity(prev_obs, obs)

    B_F_g = rot.apply(I_F_g)
    B_F_1 = B_F_a1.dot(B_w) * airspeed

    B_F_2 = B_F_a2.dot(act) * (airspeed**2)

    return B_F_1 + B_F_2 + B_F_g

In [182]:
# Estimate next grouznd speed:
def get_get_next_ground_speed(prev_obs, obs, actions):

    next_v_g = np.zeros((3))
    time_step = 0.3
    mass = 2.55

    B_v = get_ground_velocity(obs)
    B_w = get_ground_angular_velocity(prev_obs, obs)
    B_F = get_net_cg_forces(prev_obs, obs, actions)

    next_v_g = B_v + time_step * (B_F / mass + np.cross(B_w,B_v))

    return next_v_g

In [181]:
# Obtain next altitude z:
def get_next_altitude(prev_obs, obs):

    z = obs[0]
    time_step = 0.3
    v = get_next_ground_speed(prev_obs, obs, actions)

    return z + time_step * v[2]

In [161]:
# Get next airspeed:
def get_next_airspeed(prev_obs, obs, actions):
    next_v_g = get_next_ground_speed(prev_obs, obs, actions)

    z = obs[0]

    return np.linalg.norm(next_v_g - get_wind(z))

In [191]:
# Get torqe:
def get_net_torque(prev_obs, obs, actions):
    B_M_a1 = np.array([[-0.386, 0.0, 0.039],
                      [0.0, -0.066, 0.0],
                      [0.008, 0.0, -0.046]])

    B_M_a2 = np.array([[0.0, -0.005, 0.017, -0.017, 0.0, 0.001, 0.0],
                      [-0.157, 0.0, 0.001, 0.001, -0.017, 0.0, 0.002],
                      [0.0, 0.16, -0.001, 0.001, 0.0, -0.018, 0.0]])

    alpha = 0.0
    beta = 0.0

    p = actions[1]
    q = actions[0]
    r = actions[2]

    act = np.array([alpha, beta, p, q, r, 0.0, 1.0])
    airspeed = obs[1]

    B_w = get_ground_angular_velocity(prev_obs, obs)

    B_M = B_M_a1.dot(B_w) * airspeed + B_M_a2.dot(act) + (airspeed**2)

    
    return B_M

In [194]:
# Get next orientation:
def get_next_pose(prev_obs, obs, actions):
    J = np.array([[0.6688, 0.0, 0.0281],
                 [0.0, 0.16235, 0.0],
                 [0.0281, 0.0, 0.69364]])

    yaw = obs[2]
    pitch = obs[3]
    roll = obs[4]
    time_step = 0.3
    
    B_M = get_net_torque(prev_obs, obs, actions)
    B_w = get_ground_angular_velocity(prev_obs, obs)

    next_w = B_w + time_step * (np.linalg.inv(J).dot(B_M + np.cross(B_w, J.dot(B_w))))

    next_pose = np.array([yaw, pitch, roll]) + time_step * B_w

    return next_pose[0], next_pose[1], next_pose[2]
